# Calibrating the global orientation of the TART

Author: Tim Molteno tim@elec.ac.nz

We use google maps to do this and find an arm that aligns closely to a distant landmark.

In [1]:
import numpy as np

from astropy.coordinates import EarthLocation, position_angle

In [2]:
'''
const y = Math.sin(λ2-λ1) * Math.cos(φ2);
const x = Math.cos(φ1)*Math.sin(φ2) -
          Math.sin(φ1)*Math.cos(φ2)*Math.cos(λ2-λ1);
const θ = Math.atan2(y, x);
const brng = (θ*180/Math.PI + 360) % 360; // in degrees
'''

def angleFromCoordinate( lat1,  lon1,  lat2, lon2):

    dLon = np.radians(lon2 - lon1)

    l1r = np.radians(lat1)
    l2r = np.radians(lat2)
    
    y = np.sin(dLon) * np.cos(l2r)
    x = np.cos(l1r) * np.sin(l2r) - np.sin(l1r)*np.cos(l2r) \
        * np.cos(dLon)

    brng = np.arctan2(y, x)

    brng = np.degrees(brng)
    brng = (brng + 360) % 360;
    brng = 360 - brng; # count degrees counter-clockwise - remove to make clockwise

    return brng

In [3]:
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

def bearing(lat1,  lon1,  lat2, lon2):
    fwd_azimuth,back_azimuth,distance = geodesic.inv(lon1, lat1, lon2, lat2)
    return fwd_azimuth

def bearing_latlon(a, b):
    return bearing(lon1=a[1], lat1=a[0], lon2=b[1], lat2=b[0])

def get_rotation_angle(expected_angle, actual_angle):
    print(f"expected angle {expected_angle}")
    print(f"actual angle {actual_angle}")
    rotation =  -(actual_angle - expected_angle)
    return rotation

In [4]:
# Rhodes Site
tree = [-33.314532, 26.504503];
monument = [-33.319285, 26.519030]
turbine = [-33.326303, 26.469101] 
rhodes_tart = [-33.319449, 26.507807]; 

ang_tart_turbine = bearing_latlon(rhodes_tart, turbine)

# positive angles point west of geographic north. (clockwise)
ang_tart_turbine

-101.92133948345194

In [5]:
expected_angle =  -72
actual_angle = ang_tart_turbine
get_rotation_angle(expected_angle, ang_tart_turbine)

expected angle -72
actual angle -101.92133948345194


29.921339483451945

## Calculating the rotation angle

We need to find the angle that the array is rotated from geographic north. Start with the expected angle (positive angles point west of geographic north). In this case it is arm 5 that is expected to be 72 degrees

In [6]:
expected_angle =  - (72)
print(f"expected angle {expected_angle}")
actual_angle = (ang_tart_turbine)
print(f"actual angle {actual_angle}")
rotation =  -(actual_angle - expected_angle)
rotation

expected angle -72
actual angle -101.92133948345194


29.921339483451945

Now enter this information into the global_rotation field in the site survey notebook

In [9]:
# UdM site
udm_tart = [-20.2587508, 57.75919894]
udm_pole = [-20.257990, 57.760779]

ang_tart_pole = bearing_latlon(udm_tart, udm_pole)

# positive angles point west of geographic north. (clockwise)
ang_tart_pole

62.96852731799942

In [14]:
# This is with antenna 4
ant4 = [1.6261960453222395, 0.7033414756277336, 0.0]
x,y,z = ant4
expected_angle = np.arctan2(y,x)
np.degrees(expected_angle)

23.388829280943725

In [17]:
expected_angle =  23.38
get_rotation_angle(expected_angle, ang_tart_pole)

expected angle 23.38
actual angle 62.96852731799942


-39.58852731799942